In [137]:
# Initial imports.
import pandas as pd
import numpy as np
import datetime as dt
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [138]:
# Loading data after database join of seasons/quarters file and Apple's stock file
# https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e#:~:text=By%20default%2C%20date%20columns%20are%20parsed%20using%20the,a%20different%20date%20format%2C%20for%20example%2C%20YYYY-DD-MM%20HH%3AMM%3ASS%3A
file_path =Path("./AAPL_Pre_Official_DB_7_31.csv")
apple_df = pd.read_csv(file_path, parse_dates=['Date'])
apple_df.head()

,Date,Day_Month,Year,Month,Season,Quarter,Open,High,Low,Close,...,Open_Change,High_Change,Low_Change,Close_Change,Volume_Change,Gain_Loss_Open,Gain_Loss_High,Gain_Loss_Low,Gain_Loss_Close,Gain_Loss_Volume
0,1980-12-12,12/12,1980,12,Winter,Q4,0.128348,0.128906,0.128348,0.128348,...,0.128348,0.128906,0.128348,0.128348,0,Gain,Gain,Gain,Gain,Gain
1,1980-12-15,12/15,1980,12,Winter,Q4,0.122210,0.122210,0.121652,0.121652,...,-0.006138,-0.006696,-0.006696,-0.006696,-293148800,Loss,Loss,Loss,Loss,Loss
2,1980-12-16,12/16,1980,12,Winter,Q4,0.113281,0.113281,0.112723,0.112723,...,-0.008929,-0.008929,-0.008929,-0.008929,-70156800,Loss,Loss,Loss,Loss,Loss
3,1980-12-17,12/17,1980,12,Winter,Q4,0.115513,0.116071,0.115513,0.115513,...,0.002232,0.002790,0.002790,0.002790,-19286400,Gain,Gain,Gain,Gain,Loss
4,1980-12-18,12/18,1980,12,Winter,Q4,0.118862,0.119420,0.118862,0.118862,...,0.003349,0.003349,0.003349,0.003349,-12992000,Gain,Gain,Gain,Gain,Loss


In [139]:
#apple_adj_df = apple_df.drop(['Open_Change','High_Change','Low_Change','Close_Change','Gain_Loss_High','Gain_Loss_Open','Gain_Loss_Low','Gain_Loss_Close'], axis=1)
apple_adj_df = apple_df.drop(['Day_Month','Gain_Loss_High','Gain_Loss_Open','Gain_Loss_Low','Gain_Loss_Close'], axis=1)
#apple_adj_df = apple_df.drop(['Open_Change','High_Change','Low_Change','Close_Change'], axis=1)
#apple_adj_df = apple_df.drop(['Open_Change','High_Change','Low_Change','Close_Change','Gain_Loss_High','Gain_Loss_Open','Gain_Loss_Low','Gain_Loss_Close'], axis=1)
apple_adj_df.head()

,Date,Year,Month,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_Change,High_Change,Low_Change,Close_Change,Volume_Change,Gain_Loss_Volume
0,1980-12-12,1980,12,Winter,Q4,0.128348,0.128906,0.128348,0.128348,0.100751,469033600,0.128348,0.128906,0.128348,0.128348,0,Gain
1,1980-12-15,1980,12,Winter,Q4,0.122210,0.122210,0.121652,0.121652,0.095495,175884800,-0.006138,-0.006696,-0.006696,-0.006696,-293148800,Loss
2,1980-12-16,1980,12,Winter,Q4,0.113281,0.113281,0.112723,0.112723,0.088485,105728000,-0.008929,-0.008929,-0.008929,-0.008929,-70156800,Loss
3,1980-12-17,1980,12,Winter,Q4,0.115513,0.116071,0.115513,0.115513,0.090676,86441600,0.002232,0.002790,0.002790,0.002790,-19286400,Loss
4,1980-12-18,1980,12,Winter,Q4,0.118862,0.119420,0.118862,0.118862,0.093304,73449600,0.003349,0.003349,0.003349,0.003349,-12992000,Loss


In [140]:
apple_adj_df['Date'] = pd.to_datetime(apple_adj_df['Date'])
apple_adj_df['Date'] = apple_adj_df['Date'].map(dt.datetime.toordinal)
apple_adj_df.head()

,Date,Year,Month,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_Change,High_Change,Low_Change,Close_Change,Volume_Change,Gain_Loss_Volume
0,723161,1980,12,Winter,Q4,0.128348,0.128906,0.128348,0.128348,0.100751,469033600,0.128348,0.128906,0.128348,0.128348,0,Gain
1,723164,1980,12,Winter,Q4,0.122210,0.122210,0.121652,0.121652,0.095495,175884800,-0.006138,-0.006696,-0.006696,-0.006696,-293148800,Loss
2,723165,1980,12,Winter,Q4,0.113281,0.113281,0.112723,0.112723,0.088485,105728000,-0.008929,-0.008929,-0.008929,-0.008929,-70156800,Loss
3,723166,1980,12,Winter,Q4,0.115513,0.116071,0.115513,0.115513,0.090676,86441600,0.002232,0.002790,0.002790,0.002790,-19286400,Loss
4,723167,1980,12,Winter,Q4,0.118862,0.119420,0.118862,0.118862,0.093304,73449600,0.003349,0.003349,0.003349,0.003349,-12992000,Loss


In [141]:
# Convert date into an integer since the system was not taking it as a float so that the ML model can read the date - for now.
# https://www.geeksforgeeks.org/convert-floats-to-integers-in-a-pandas-dataframe/
#apple_df['Date'] = apple_df['Date'].astype(int)

In [142]:
# Must remain in order to activate label_binarize
#If in doubt refer to sklearn
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
apple_gain_loss_df = apple_adj_df.copy()
apple_gain_loss_df['Gain_Loss_Volume'] = le.fit_transform(apple_gain_loss_df['Gain_Loss_Volume'])
apple_gain_loss_df.head()

,Date,Year,Month,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_Change,High_Change,Low_Change,Close_Change,Volume_Change,Gain_Loss_Volume
0,723161,1980,12,Winter,Q4,0.128348,0.128906,0.128348,0.128348,0.100751,469033600,0.128348,0.128906,0.128348,0.128348,0,0
1,723164,1980,12,Winter,Q4,0.122210,0.122210,0.121652,0.121652,0.095495,175884800,-0.006138,-0.006696,-0.006696,-0.006696,-293148800,1
2,723165,1980,12,Winter,Q4,0.113281,0.113281,0.112723,0.112723,0.088485,105728000,-0.008929,-0.008929,-0.008929,-0.008929,-70156800,1
3,723166,1980,12,Winter,Q4,0.115513,0.116071,0.115513,0.115513,0.090676,86441600,0.002232,0.002790,0.002790,0.002790,-19286400,1
4,723167,1980,12,Winter,Q4,0.118862,0.119420,0.118862,0.118862,0.093304,73449600,0.003349,0.003349,0.003349,0.003349,-12992000,1


In [143]:
# If in doubt look up scikit learn label_binarize
from sklearn.preprocessing import label_binarize

binarized_gain_loss = label_binarize(y=list(apple_gain_loss_df['Gain_Loss_Volume']), classes=[1,0])
apple_gain_loss_df = apple_adj_df.copy()
apple_gain_loss_df['Gain_Loss_Volume'] = binarized_gain_loss
apple_gain_loss_df.head()

,Date,Year,Month,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_Change,High_Change,Low_Change,Close_Change,Volume_Change,Gain_Loss_Volume
0,723161,1980,12,Winter,Q4,0.128348,0.128906,0.128348,0.128348,0.100751,469033600,0.128348,0.128906,0.128348,0.128348,0,1
1,723164,1980,12,Winter,Q4,0.122210,0.122210,0.121652,0.121652,0.095495,175884800,-0.006138,-0.006696,-0.006696,-0.006696,-293148800,0
2,723165,1980,12,Winter,Q4,0.113281,0.113281,0.112723,0.112723,0.088485,105728000,-0.008929,-0.008929,-0.008929,-0.008929,-70156800,0
3,723166,1980,12,Winter,Q4,0.115513,0.116071,0.115513,0.115513,0.090676,86441600,0.002232,0.002790,0.002790,0.002790,-19286400,0
4,723167,1980,12,Winter,Q4,0.118862,0.119420,0.118862,0.118862,0.093304,73449600,0.003349,0.003349,0.003349,0.003349,-12992000,0


In [144]:
# Perform binary encoding of Season and Quarter columns
apple_binary_encoded = pd.get_dummies(apple_gain_loss_df, columns=["Season", "Quarter"])
apple_binary_encoded.head()

,Date,Year,Month,Open,High,Low,Close,Adj Close,Volume,Open_Change,...,Volume_Change,Gain_Loss_Volume,Season_Fall,Season_Spring,Season_Summer,Season_Winter,Quarter_Q1,Quarter_Q2,Quarter_Q3,Quarter_Q4
0,723161,1980,12,0.128348,0.128906,0.128348,0.128348,0.100751,469033600,0.128348,...,0,1,0,0,0,1,0,0,0,1
1,723164,1980,12,0.122210,0.122210,0.121652,0.121652,0.095495,175884800,-0.006138,...,-293148800,0,0,0,0,1,0,0,0,1
2,723165,1980,12,0.113281,0.113281,0.112723,0.112723,0.088485,105728000,-0.008929,...,-70156800,0,0,0,0,1,0,0,0,1
3,723166,1980,12,0.115513,0.116071,0.115513,0.115513,0.090676,86441600,0.002232,...,-19286400,0,0,0,0,1,0,0,0,1
4,723167,1980,12,0.118862,0.119420,0.118862,0.118862,0.093304,73449600,0.003349,...,-12992000,0,0,0,0,1,0,0,0,1


In [145]:
# Define features set
X = apple_binary_encoded.copy()
X = X.drop(["Gain_Loss_Volume","Volume_Change"], axis=1)
X.head()

,Date,Year,Month,Open,High,Low,Close,Adj Close,Volume,Open_Change,...,Low_Change,Close_Change,Season_Fall,Season_Spring,Season_Summer,Season_Winter,Quarter_Q1,Quarter_Q2,Quarter_Q3,Quarter_Q4
0,723161,1980,12,0.128348,0.128906,0.128348,0.128348,0.100751,469033600,0.128348,...,0.128348,0.128348,0,0,0,1,0,0,0,1
1,723164,1980,12,0.122210,0.122210,0.121652,0.121652,0.095495,175884800,-0.006138,...,-0.006696,-0.006696,0,0,0,1,0,0,0,1
2,723165,1980,12,0.113281,0.113281,0.112723,0.112723,0.088485,105728000,-0.008929,...,-0.008929,-0.008929,0,0,0,1,0,0,0,1
3,723166,1980,12,0.115513,0.116071,0.115513,0.115513,0.090676,86441600,0.002232,...,0.002790,0.002790,0,0,0,1,0,0,0,1
4,723167,1980,12,0.118862,0.119420,0.118862,0.118862,0.093304,73449600,0.003349,...,0.003349,0.003349,0,0,0,1,0,0,0,1


In [146]:
# Define the target set.
y = apple_df["Gain_Loss_Volume"].ravel()
y[:5]

array(['Gain', 'Loss', 'Loss', 'Loss', 'Loss'], dtype=object)

In [147]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.80)

In [148]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [149]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=100, random_state=78) 

In [150]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [151]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [152]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,698,292
Actual 1,275,781


In [153]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [154]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,698,292
Actual 1,275,781


Accuracy Score : 0.7228739002932552
Classification Report
              precision    recall  f1-score   support

        Gain       0.72      0.71      0.71       990
        Loss       0.73      0.74      0.73      1056

    accuracy                           0.72      2046
   macro avg       0.72      0.72      0.72      2046
weighted avg       0.72      0.72      0.72      2046



In [155]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.07851206, 0.03387524, 0.03164298, 0.06153777, 0.05940832,
       0.0594561 , 0.05918624, 0.06084721, 0.16170906, 0.07373502,
       0.09572181, 0.09675505, 0.07853484, 0.00629208, 0.00645536,
       0.00630625, 0.00609825, 0.0054317 , 0.0063913 , 0.00667294,
       0.00543042])

In [156]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.1617090607650535, 'Volume'),
 (0.09675505261711584, 'Low_Change'),
 (0.09572181132431737, 'High_Change'),
 (0.07853483581002313, 'Close_Change'),
 (0.07851205899522744, 'Date'),
 (0.0737350172081163, 'Open_Change'),
 (0.06153776798909261, 'Open'),
 (0.060847210748287195, 'Adj Close'),
 (0.05945609729719178, 'Low'),
 (0.05940832245464674, 'High'),
 (0.059186240116383716, 'Close'),
 (0.03387524140966995, 'Year'),
 (0.03164298492238956, 'Month'),
 (0.006672937003362517, 'Quarter_Q3'),
 (0.006455363259857279, 'Season_Spring'),
 (0.006391296252391685, 'Quarter_Q2'),
 (0.006306254787363575, 'Season_Summer'),
 (0.0062920768788472, 'Season_Fall'),
 (0.006098246754797705, 'Season_Winter'),
 (0.0054317031649238744, 'Quarter_Q1'),
 (0.00543042024094089, 'Quarter_Q4')]